In [ ]:
import freesound
import os

# Initialize the Freesound client
client = freesound.FreesoundClient()
client.set_token("OBPGQjX4GuhWK310ErmBtz1HIRBCBsv29sfQqszV")

# Define a list of keywords relevant to smart speakers and IoT
keywords = [
    "alarm", "notification", "doorbell", "chime", "alert", "timer",
    "command", "beep", "ringtone", "announcement", "music", "weather",
    "traffic", "reminder", "message", "phone", "email", "calendar",
    "meeting", "reminder", "cooking", "cleaning", "security", "lock", "unlock"
]

# Create a folder to store the downloaded sounds
output_folder = "/external2/datasets/noise/smart_speaker_sounds"
os.makedirs(output_folder, exist_ok=True)

# Function to sanitize filenames by replacing spaces and slashes with underscores
def sanitize_filename(filename):
    return filename.replace(" ", "_").replace("/", "_")

# Loop through the keywords and download the relevant sounds
for keyword in keywords:
    results = client.text_search(query=keyword, fields="id,name,previews", page_size=10)
    for sound in results:
        sanitized_name = sanitize_filename(f"{keyword}_{sound.name}.mp3")
        sound.retrieve_preview(output_folder, sanitized_name)
        print(f"Downloaded {sanitized_name} for keyword {keyword}")


In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import time
import requests

# Set the URL of the site
url = "https://www.freesoundeffects.com/"

# Create a directory to save the files
download_dir = "sound_effects"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Function to download a file from a URL
def download_file(url, download_dir):
    local_filename = os.path.join(download_dir, url.split('/')[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

# Set up Selenium and open the website
options = webdriver.FirefoxOptions()
options.add_argument("--headless")  # Run headless Firefox
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
driver.get(url)

# Wait for the page to load completely
try:
    # Wait until the links are present
    WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
    print("Page loaded successfully.")
except Exception as e:
    print("Error: Page did not load successfully.")
    driver.quit()
    exit(1)

# Find all download links
links = driver.find_elements(By.TAG_NAME, "a")
print(links)

wav_links = []
for link in links:
    href = link.get_attribute("href")
    if href and href.endswith(".wav"):
        wav_links.append(href)

# Download each WAV file
for wav_link in wav_links:
    print(f"Downloading {wav_link}...")
    try:
        download_file(wav_link, download_dir)
        print(f"Downloaded {wav_link}")
    except Exception as e:
        print(f"Failed to download {wav_link}: {e}")

# Close the browser
driver.quit()


In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
import time
import requests

# Set the URL of the site
url = "https://www.freesoundeffects.com/"

# Create a directory to save the files
download_dir = "sound_effects"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Function to download a file from a URL
def download_file(url, download_dir):
    local_filename = os.path.join(download_dir, url.split('/')[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

# Set up Selenium and open the website
options = webdriver.FirefoxOptions()
options.add_argument("--headless")  # Run headless Firefox
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
driver.get(url)

# Wait for the page to load
#time.sleep(5)

# Find all download links
links = driver.find_elements(By.TAG_NAME, "a")
print(links)

# wav_links = []
# for link in links:
#     href = link.get_attribute("href")
#     if href and href.endswith(".wav"):
#         wav_links.append(href)

# # Download each WAV file
# for wav_link in wav_links:
#     print(f"Downloading {wav_link}...")
#     try:
#         download_file(wav_link, download_dir)
#         print(f"Downloaded {wav_link}")
#     except Exception as e:
#         print(f"Failed to download {wav_link}: {e}")

# # Close the browser
# driver.quit()
